In [1]:
# L'exécution de cette cellule démarre l'application Spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1697746103709_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### I. Affichage des informations sur la session en cours et liens vers Spark UI

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1697746103709_0001,pyspark,idle,Link,Link,✔


### II. Installation des packages

Les packages nécessaires ont été installés via l'étape de **boostrap** à l'instantiation du serveur. 
### III. Import des librairies : 

In [3]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.10.4 Définition des PATH pour charger les images et enregistrer les résultats

Nous accédons directement à nos **données sur S3** comme si elles étaient **stockées localement**.

In [5]:
PATH = 's3://p8-oc-data-test'
PATH_Data = PATH+'/Test1'
PATH_Result = PATH+'/Results'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PATH:        s3://p8-oc-data-test
PATH_Data:   s3://p8-oc-data-test/Test1
PATH_Result: s3://p8-oc-data-test/Results

### 4.10.5 Traitement des données

#### 4.10.5.1 Chargement des données

In [6]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://p8-oc-data-t...|2023-10-19 20:42:10|  5464|[FF D8 FF E0 00 1...|
|s3://p8-oc-data-t...|2023-10-19 20:42:10|  5421|[FF D8 FF E0 00 1...|
|s3://p8-oc-data-t...|2023-10-19 20:42:12|  5420|[FF D8 FF E0 00 1...|
|s3://p8-oc-data-t...|2023-10-19 20:42:13|  5416|[FF D8 FF E0 00 1...|
|s3://p8-oc-data-t...|2023-10-19 20:42:12|  5401|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

<u>Je ne conserve que le **path** de l'image et j'ajoute <br />
    une colonne contenant les **labels** de chaque image</u> :

In [8]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+-------------------------------------------+------+
|path                                       |label |
+-------------------------------------------+------+
|s3://p8-oc-data-test/Test1/Orange/3_100.jpg|Orange|
|s3://p8-oc-data-test/Test1/Orange/4_100.jpg|Orange|
|s3://p8-oc-data-test/Test1/Orange/7_100.jpg|Orange|
|s3://p8-oc-data-test/Test1/Orange/8_100.jpg|Orange|
|s3://p8-oc-data-test/Test1/Orange/6_100.jpg|Orange|
+-------------------------------------------+------+
only showing top 5 rows

None

#### 4.10.5.2 Préparation du modèle

In [9]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14540800/14536120 [==============================] - 1s 0us/step

In [10]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [13]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

#### 4.10.5.3 Définition du processus de chargement des images <br/> et application de leur featurisation à travers l'utilisation de pandas UDF

In [14]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

#### 4.10.5.4 Exécutions des actions d'extractions de features

In [15]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
print(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://p8-oc-data-test/Results

In [18]:
features_df.write.mode("overwrite").parquet(PATH_Result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4.10.6 Chargement des données enregistrées et validation du résultat

In [19]:
parquet_df=spark.read.parquet(PATH_Result)
#df = pd.read_parquet(PATH_Result, engine='pyarrow')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
parquet_df.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+--------------------+
|                path|        label|            features|
+--------------------+-------------+--------------------+
|s3://p8-oc-data-t...|        Peach|[1.2615149, 0.0, ...|
|s3://p8-oc-data-t...|Passion Fruit|[0.2271268, 0.0, ...|
|s3://p8-oc-data-t...|  Onion White|[0.1631264, 0.258...|
|s3://p8-oc-data-t...|Passion Fruit|[0.26558402, 0.0,...|
|s3://p8-oc-data-t...|        Peach|[1.5029683, 0.0, ...|
+--------------------+-------------+--------------------+
only showing top 5 rows

In [21]:
df=parquet_df.toPandas()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                                path  ...                                           features
0         s3://p8-oc-data-test/Test1/Peach/7_100.jpg  ...  [1.2615149021148682, 0.0, 0.0, 0.0, 0.04476949...
1  s3://p8-oc-data-test/Test1/Passion Fruit/63_10...  ...  [0.227126806974411, 0.0, 0.0, 0.0, 0.033640149...
2  s3://p8-oc-data-test/Test1/Onion White/89_100.jpg  ...  [0.1631263941526413, 0.25855961441993713, 0.0,...
3  s3://p8-oc-data-test/Test1/Passion Fruit/64_10...  ...  [0.2655840218067169, 0.0, 0.0, 0.0, 0.08658794...
4         s3://p8-oc-data-test/Test1/Peach/9_100.jpg  ...  [1.5029683113098145, 0.0, 0.0, 0.0, 0.01452823...

[5 rows x 3 columns]

In [23]:
df.shape

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(65, 3)

In [24]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import PCA
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.types import ArrayType
from pyspark.sql.types import DoubleType

# Créez un DataFrame Spark à partir du DataFrame Pandas
spark_df = spark.createDataFrame(df)

# Définissez une UDF pour convertir les tableaux NumPy en Vector
def numpy_to_vector(arr):
    return Vectors.dense(arr)

udf_numpy_to_vector = udf(numpy_to_vector, VectorUDT())

# Appliquez l'UDF pour convertir la colonne "features"
spark_df = spark_df.withColumn('features', udf_numpy_to_vector(spark_df['features']))

# Appliquez PCA sur la colonne "features"
pca = PCA(k=100, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(spark_df)
pca_result = pca_model.transform(spark_df)

# Affichez la variance expliquée
print(pca_model.explainedVariance)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.07611281066046868,0.06434559297221037,0.05840154795728417,0.055224641959716644,0.05262386647037515,0.051968914642221045,0.049868474122884525,0.043698043581583766,0.041108503673628255,0.04047390610505519,0.038430791798938754,0.037067465160487194,0.03507168672328703,0.03389232614656543,0.029910288065219216,0.027268272535885922,0.02536808313049681,0.02532286090065227,0.022990663570425444,0.02268237729166185,0.021357794055699975,0.020191622365275597,0.018450331607690874,0.017767353765112865,0.01720222898011078,0.014069424374378555,0.012120064470131375,0.011368004595410987,0.005941530196595519,0.004430781446890578,0.0034069453751330316,0.0024168178130842015,0.0019981788597209877,0.0016524679197123012,0.0014894746427993497,0.0012793276572115964,0.0011251963139669021,0.0010306619705939292,0.0010006131239814417,0.0009240370816789386,0.0008198668152771667,0.000745764339340905,0.0007231248311901412,0.0006396067635780054,0.000603022256055364,0.0005620007209951647,0.0005521777625944576,0.000529

In [35]:
# Sélectionnez uniquement les colonnes "path" et "pca_features" du DataFrame pca_result
pca_result_selected = pca_result.select("path", "pca_features")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
# Spécifiez le chemin de destination dans votre bucket S3
s3_output_path = 's3://p8-oc-data-test/output.csv'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
from pyspark.sql.functions import col, expr

# Convertir la colonne "pca_features" en une représentation textuelle
pca_result_text = pca_result.withColumn("pca_features", expr("cast(pca_features as string)"))

# Sélectionner uniquement les colonnes "path" et "pca_features"
pca_result_selected = pca_result_text.select("path", "pca_features")

# Spécifier le chemin de destination dans votre bucket S3
s3_output_path = 's3://p8-oc-data-test/output.csv'

# Enregistrer le DataFrame sélectionné au format CSV dans S3
pca_result_selected.write.option("header", "true").csv(s3_output_path)

print(f"Le fichier CSV a été enregistré dans {s3_output_path}")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Le fichier CSV a été enregistré dans s3://p8-oc-data-test/output.csv